# Matrix Factorization
In "UNKs Everywhere: Adapting Multilingual Language Models to New Scripts" by Pfeiffer et al. (2021) they describe a method factorize the embedding matrix **X** of a pre-trained language model into two smaller matrices:

- **F**: This matrix represents new, lower-dimensional word embeddings (D' = 100). It contains token-specific information.
- **G**: This represents up-projection matrices that help in reconstructing the original embeddings from **F**. This encodes general properties of the original embedding matrix.
  - In the most basic case **X** is approximately **FG**, which means that **G** is shared across all tokens and encodes general properties of the original embedding matrix whereas **F** stores token-specific information. **G** only needs to be pretrained once and can be used and fine-tuned for every new language. Only need to learn new low-dimensional embeddings **F** with pretraining task.
  - C up-projection matrices: group tokens and use separate up-projection matrix per group to balance sharing information across typologically similar languages with learning a robust representation for each token
    - K-MEANS to cluster X into C clusters, factorize subset of embeddings associated with c-th cluster separately. For new language randomly initialize a matrix in order to compute cluster assignment matrix with softmax estimation
    - NEURAL learns cluster assignment and up-projections jointly by parameterizing **G**

When adapting to a new language, instead of starting from scratch, the method initializes the new embedding matrix using these lower-dimensional embeddings and up-projections. 
By doing so, it uses the general linguistic knowledge already present in the original model (encoded in **G**) and combines it with the specifics of the target language (captured during the adaptation process in **F**).
The lower-dimensional word embedding **F'** of the new model is either randomly initialized or partially initialized using the embeddings in **F** of overlapping tokens between the source and the target language.

For cross-lingual transfer only language adapters and embeddings **F** are trained jointly while the rest of the model (including **G**) is fixed. This further reduces the number of trainable parameters.

They formulate this as a semi non-negative matrix factorization problem.
They reference this code base: https://github.com/cthurau/pymf/tree/master/pymf

That code base is quite old. We copied the relevant parts of that code base and updated it to work with our code.

In [1]:
import numpy as np
from ..src.utils.snmf import SNMF


# Example Usage
vocab_size, original_dim = 3000, 768  # Example dimensions
x = np.random.randn(vocab_size, original_dim)  # Simulated embedding matrix
keep_dim = 100
snmf_mdl = SNMF(x, niter=3000, num_bases=keep_dim)
# Factorizes into 
# W - low-dimensional word embeddings F (3000 x 100) and 
# H - up-projections G (100 x 768)
snmf_mdl.factorize()
f = snmf_mdl.W
g = snmf_mdl.H
print(f"Lower-dimensional word embeddings F shape: {f.shape}")
# TODO K-means clustering/neural clustering for word embeddings

Lower-dimensional word embeddings F shape: (3000, 100)


Whereas Pfeiffer et al. (2021) formulate it as semi non-negative matrix factorization, in "OFA: A Framework of Initializing Unseen Subword Embeddings for Efficient Large-scale Multilingual Continued Pretraining" Liu et al. (2024) use Singular Value Decomposition (SVD) to factorize the word embedding matrix into a lower-dimensional (lower coordinates) matrix and an orthogonal up-projection matrix (primitive embeddings, language agnostic).

In [2]:
import numpy as np

# Example Usage
vocab_size, original_dim = 3000, 768  # Example dimensions
keep_dim = 100
x = np.random.randn(vocab_size, original_dim)  # Simulated embedding matrix

try:
    # factorize the multilingual embedding using svd
    u, s, vh = np.linalg.svd(x, full_matrices=True)

    primitive_embeddings = np.matmul(vh.T[:, :keep_dim], np.diag(s[:keep_dim])).T
    # primitive_embeddings size: (keep_dim, vector_size)

    lower_coordinates = u[:, :keep_dim]
    # size: (num_words, keep_dim)
    print(f"Lower-dimensional word embeddings shape: {lower_coordinates.shape}")
except:
    raise ValueError("Cannot perform the factorization!")

Lower-dimensional word embeddings shape: (3000, 100)


In [3]:
print(f"Primitive embeddings shape: {primitive_embeddings.shape}")

Primitive embeddings shape: (100, 768)


My questions:

- U and V transposed were orthogonal (orthonormal) matrices with the left/right singular vectors. Do their reduced versions retain their properties?
- Is the product of reduced Sigma and reduced V transposed orthogonal?

In [4]:
np.dot(u.T, u)

array([[ 1.00000000e+00,  3.89228580e-17, -3.98986399e-17, ...,
        -2.25514052e-17,  1.90819582e-17,  1.38777878e-17],
       [ 3.89228580e-17,  1.00000000e+00, -1.76508114e-16, ...,
        -2.08166817e-17, -2.34187669e-17, -1.21430643e-17],
       [-3.98986399e-17, -1.76508114e-16,  1.00000000e+00, ...,
        -5.20417043e-18, -6.93889390e-18, -2.94902991e-17],
       ...,
       [-2.25514052e-17, -2.08166817e-17, -5.20417043e-18, ...,
         1.00000000e+00,  1.04083409e-17,  1.04083409e-17],
       [ 1.90819582e-17, -2.34187669e-17, -6.93889390e-18, ...,
         1.04083409e-17,  1.00000000e+00, -1.04083409e-17],
       [ 1.38777878e-17, -1.21430643e-17, -2.94902991e-17, ...,
         1.04083409e-17, -1.04083409e-17,  1.00000000e+00]])

In order to to understand the differences of the matrix factorization methods, we will take a look at the example in Dinh et al. (2008).

In [19]:
data_matrix = np.matrix([
    [1.3, 1.8, 4.8, 7.1, 5.0, 5.2, 8.0],
    [1.5, 6.9, 3.9, -5.5, -8.5, -3.9, -5.5],
    [6.5, 1.6, 8.2, -7.2, -8.7, -7.9, -5.2],
    [3.8, 8.3, 4.7, 6.4, 7.5, 3.2, 7.4],
    [-7.3, -1.8, -2.1, 2.7, 6.8, 4.8, 6.2],
])

In [20]:
data_matrix

matrix([[ 1.3,  1.8,  4.8,  7.1,  5. ,  5.2,  8. ],
        [ 1.5,  6.9,  3.9, -5.5, -8.5, -3.9, -5.5],
        [ 6.5,  1.6,  8.2, -7.2, -8.7, -7.9, -5.2],
        [ 3.8,  8.3,  4.7,  6.4,  7.5,  3.2,  7.4],
        [-7.3, -1.8, -2.1,  2.7,  6.8,  4.8,  6.2]])

In [26]:
u, s, v = np.linalg.svd(data_matrix.T)

In [27]:
u

matrix([[ 0.19570959, -0.46558784, -0.49758201,  0.58923896,  0.00263719,
          0.34827514,  0.1698022 ],
        [ 0.03485142, -0.57554339,  0.78543451,  0.19257571, -0.09009707,
         -0.02563702,  0.06904831],
        [ 0.13145959, -0.60549496, -0.26247475, -0.59310945,  0.03866713,
         -0.0413909 , -0.43842224],
        [-0.45364988, -0.16907824, -0.1331825 ,  0.31560236,  0.48312969,
         -0.62687319, -0.14794463],
        [-0.57682319, -0.02287634, -0.07311434,  0.17287702, -0.67945481,
          0.09704743, -0.40055269],
        [-0.40581001,  0.0346385 ,  0.15872702, -0.05175672,  0.53645674,
          0.6880708 , -0.21217472],
        [-0.48989441, -0.2346125 , -0.13540392, -0.36299195, -0.08672618,
          0.02245745,  0.73948448]])

In [28]:
s

array([28.4480211 , 16.96105118,  6.75969418,  5.16119986,  3.28045566])

In [29]:
v

matrix([[-0.39321668,  0.44719741,  0.57803086, -0.36918282, -0.41830549],
        [-0.44568199, -0.28013616, -0.38615063, -0.72348304,  0.22439091],
        [-0.30503919,  0.75878217, -0.45633945,  0.22188508,  0.27151801],
        [-0.34917987, -0.2145735 , -0.42727545,  0.29344783, -0.75057025],
        [ 0.65709414,  0.31582597, -0.35490563, -0.45269386, -0.3709333 ]])

In [41]:
primitive_embeddings = np.matmul(v.T[:, :2], np.diag(s[:2])).T

lower_coordinates = u[:, :2]

In [42]:
primitive_embeddings

matrix([[-11.18623642,  12.7218814 ,  16.4438342 , -10.50252056,
         -11.8999635 ],
        [ -7.55923509,  -4.7514037 ,  -6.54952068, -12.27103279,
           3.80590567]])

In [43]:
lower_coordinates

matrix([[ 0.19570959, -0.46558784],
        [ 0.03485142, -0.57554339],
        [ 0.13145959, -0.60549496],
        [-0.45364988, -0.16907824],
        [-0.57682319, -0.02287634],
        [-0.40581001,  0.0346385 ],
        [-0.48989441, -0.2346125 ]])